In [6]:
dataset_path = os.path.join(path, "IDC_regular_ps50_idx5")

limit = 15000
count = 0

for patient in os.listdir(dataset_path):

    patient_path = os.path.join(dataset_path, patient)

    for label in ["0", "1"]:

        folder = os.path.join(patient_path, label)

        if not os.path.exists(folder):
            continue

        for img_name in os.listdir(folder):

            if count >= limit:
                break

            img_path = os.path.join(folder, img_name)

            img = cv2.imread(img_path)
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            img = img / 255.0

            data.append(img)
            labels.append(int(label))

            count += 1

        if count >= limit:
            break

    if count >= limit:
        break


In [8]:
import numpy as np
import os
import cv2
from sklearn.model_selection import train_test_split

IMG_SIZE = 50
data = []
labels = []

dataset_path = os.path.join(path, "IDC_regular_ps50_idx5")

limit = 15000
count = 0

for patient in os.listdir(dataset_path):

    patient_path = os.path.join(dataset_path, patient)

    for label in ["0", "1"]:

        folder = os.path.join(patient_path, label)

        if not os.path.exists(folder):
            continue

        for img_name in os.listdir(folder):

            if count >= limit:
                break

            img_path = os.path.join(folder, img_name)

            img = cv2.imread(img_path)
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            img = img / 255.0

            data.append(img)
            labels.append(int(label))

            count += 1

        if count >= limit:
            break

    if count >= limit:
        break

data = np.array(data)
labels = np.array(labels)

print("Loaded images:", len(data))

X_train, X_test, y_train, y_test = train_test_split(
    data, labels, test_size=0.2, random_state=42
)


Loaded images: 15000


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Flatten images
X_train_flat = X_train.reshape(X_train.shape[0], -1)
X_test_flat = X_test.reshape(X_test.shape[0], -1)

# Logistic Regression
lr = LogisticRegression(max_iter=1000)
lr.fit(X_train_flat, y_train)
print("LR Accuracy:", accuracy_score(y_test, lr.predict(X_test_flat)))

# KNN
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_flat, y_train)
print("KNN Accuracy:", accuracy_score(y_test, knn.predict(X_test_flat)))

# SVM
svm = SVC()
svm.fit(X_train_flat, y_train)
print("SVM Accuracy:", accuracy_score(y_test, svm.predict(X_test_flat)))


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LR Accuracy: 0.7946666666666666
KNN Accuracy: 0.8116666666666666


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Flatten, Dense, Dropout

model1 = Sequential()

model1.add(Conv2D(32, (3,3), activation='relu', input_shape=(50,50,3)))
model1.add(Conv2D(64, (3,3), activation='relu'))
model1.add(MaxPooling2D((2,2)))
model1.add(Dropout(0.25))

model1.add(Flatten())
model1.add(Dense(128, activation='relu'))
model1.add(Dropout(0.25))
model1.add(Dense(2, activation='softmax'))

model1.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model1.fit(X_train, y_train, epochs=12, batch_size=128,
           validation_split=0.2)


In [ ]:
from tensorflow.keras.layers import BatchNormalization

model2 = Sequential()

model2.add(Conv2D(32,(3,3),activation='relu',padding='same',input_shape=(50,50,3)))
model2.add(Conv2D(32,(3,3),activation='relu',padding='same'))
model2.add(MaxPooling2D((2,2)))
model2.add(BatchNormalization())
model2.add(Dropout(0.25))

model2.add(Conv2D(64,(3,3),activation='relu',padding='same'))
model2.add(Conv2D(64,(3,3),activation='relu',padding='same'))
model2.add(MaxPooling2D((2,2)))
model2.add(BatchNormalization())
model2.add(Dropout(0.25))

model2.add(Flatten())
model2.add(Dense(512,activation='relu'))
model2.add(Dropout(0.5))
model2.add(Dense(2,activation='softmax'))

model2.compile(optimizer='adam',
               loss='sparse_categorical_crossentropy',
               metrics=['accuracy'])

model2.fit(X_train, y_train, epochs=12, batch_size=128,
           validation_split=0.2)


In [ ]:
model3 = Sequential()

model3.add(Conv2D(32,(5,5),activation='relu',input_shape=(50,50,3)))
model3.add(MaxPooling2D((3,3)))

model3.add(Conv2D(32,(5,5),activation='relu'))
model3.add(MaxPooling2D((3,3)))

model3.add(Dropout(0.25))
model3.add(Flatten())
model3.add(Dense(64,activation='relu'))
model3.add(Dropout(0.5))
model3.add(Dense(2,activation='softmax'))

model3.compile(optimizer='adam',
               loss='sparse_categorical_crossentropy',
               metrics=['accuracy'])

model3.fit(X_train, y_train, epochs=12, batch_size=128,
           validation_split=0.2)


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

pred = model3.predict(X_test)
pred = np.argmax(pred, axis=1)

print(classification_report(y_test, pred))
print(confusion_matrix(y_test, pred))
